In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
mongo = MongoClient(port=27017)

In [ ]:
db = mongo['uk_food']

In [ ]:
print(db.list_collection_names())

In [ ]:
establishments = db['establishments']

In [ ]:
query = {'scores.Hygiene': 20}
print("Number of documents in result: ", establishments.count_documents(query))
print("First result:")
results = establishments.find(query)
pprint(results[0])

In [ ]:
hygienic_20 = establishments.find(query)
hygienic_20_df = pd.DataFrame(hygienic_20)
print("Documents in DataFrame: ", len(hygienic_20_df))
hygienic_20_df.head(10)

In [ ]:
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': 4}}
print("Number of documents in result: ", establishments.count_documents(query))
print("First result:")
results = establishments.find(query)
pprint(results[0])

In [ ]:
london_4s = establishments.find(query)
london_4s_df = pd.DataFrame(london_4s)
print("Documents in DataFrame: ", len(london_4s_df))
london_4s_df.head(10)

In [ ]:

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'geocode.latitude': {'$gte':latitude-degree_search, '$lte':latitude+degree_search}, 
         'geocode.longitude': {'$gte': longitude-degree_search, '$lte': longitude+degree_search},
         'RatingValue': 5}
sort = [('score.Hygiene', 1)]
limit = 5

pprint(list(establishments.find(query).sort(sort).limit(limit)))

In [ ]:
near_flavours = establishments.find(query).sort(sort).limit(limit)
near_flavours_df = pd.DataFrame(near_flavours)
near_flavours_df

In [ ]:

pipeline = [
    {'$match': {'scores.Hygiene': 0}},
    {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}},
    {'$sort': { 'count': -1 }}
]

results = list(establishments.aggregate(pipeline))

print("Number of documents in result: ", len(results))

pprint(results[0:10])

In [ ]:
result_df = pd.DataFrame(results)

print("Rows in DataFrame: ", len(result_df))
result_df.head(10)